In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
import torch
torch.set_float32_matmul_precision('medium')

In [3]:
# from datasets.shapenet.shapenet_loader import get_dataloaders
# from datasets.shapenet.shapenet_pointflow_sparse import get_dataloaders
from datasets.modelnet40.modelnet40_loader import get_dataloaders
categories = ['airplane']
# path = "../data/ShapeNetCore"
# tr, te, val = get_dataloaders(path, categories=categories)
path = "../data/ModelNet40"
# path = "../data/ShapeNetCore/pointclouds"
tr, te = get_dataloaders(path, categories=categories)

Loading renders for airplane: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4847.00it/s]


In [4]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features": [32, 64, 128, 192, 192, 256],
    "num_layers": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features": [256, 192, 192],
        "num_layers": 1,
        "attn_heads": 8,
    },
    {
        "features": [192, 128, 64, 32],
        "num_layers": 1,
        "attn_heads": None
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

from my_models.lightning_base import DiffusionBase
model = DiffusionBase(model, lr=1e-4)

In [5]:
import lightning as L

epochs = 100
trainer = L.Trainer(
    max_epochs=epochs, 
    gradient_clip_val=10.0,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 35.0 M | train
------------------------------------------
35.0 M    Trainable params
0         Non-trainable params
35.0 M    Total params
139.937   Total estimated model params size (MB)
372       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=100` reached.


In [12]:
import os
os.makedirs("../checkpoints/ModelNet40/", exist_ok=True)
torch.save(model.state_dict(), "../checkpoints/ModelNet40/airplane.ckpt")

In [10]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
ddpm_sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=1024)

model = model.cuda().eval()
preds = ddpm_sched.sample(model, 16, 2048)

Sampling: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [01:01<00:00, 16.66it/s]


In [11]:
from utils.visualization import visualize_notebook

visualize_notebook(preds, x_offset=8, y_offset=8, point_size=0.25)

Output()